In [1]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = "/home/data/train.json"

# Read the JSON file
with open(train_path, 'r') as f:
    train_data = json.load(f)

print(f"Number of training samples: {len(train_data)}")
print(f"Type of data: {type(train_data)}")
print(f"First sample keys: {list(train_data[0].keys())}")

Number of training samples: 2878
Type of data: <class 'list'>
First sample keys: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subredd

In [2]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(train_data)

# Check basic info
print("DataFrame shape:", df.shape)
print("\nTarget distribution:")
print(df['requester_received_pizza'].value_counts())
print(f"\nSuccess rate: {df['requester_received_pizza'].mean():.3f}")

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum().head(10))

DataFrame shape: (2878, 32)

Target distribution:
requester_received_pizza
False    2163
True      715
Name: count, dtype: int64

Success rate: 0.248

Missing values:
giver_username_if_known                        0
number_of_downvotes_of_request_at_retrieval    0
number_of_upvotes_of_request_at_retrieval      0
post_was_edited                                0
request_id                                     0
request_number_of_comments_at_retrieval        0
request_text                                   0
request_text_edit_aware                        0
request_title                                  0
requester_account_age_in_days_at_request       0
dtype: int64


In [ ]:
# Explore text features
print("Sample request title:")
print(df['request_title'].iloc[0])
print("\n" + "="*50)
print("Sample request text (first 500 chars):")
print(df['request_text'].iloc[0][:500])

# Check text lengths
print("\n" + "="*50)
print("Text length statistics:")
print("Title length - mean:", df['request_title'].str.len().mean(), "max:", df['request_title'].str.len().max())
print("Text length - mean:", df['request_text'].str.len().mean(), "max:", df['request_text'].str.len().max())
print("Edit-aware text length - mean:", df['request_text_edit_aware'].str.len().mean())